# Actions of Stateful Agent
Developers also could create a class and enhance its functionality using ActionWeaver's action decorators.

In [ ]:
import os
import openai
from openai import OpenAI
from actionweaver.llms import patch
from actionweaver import action
from typing import List


openai.api_key = os.getenv("OPENAI_API_KEY")

In [ ]:
class AgentV0:
    def __init__(self):
        self.llm = patch(OpenAI())
        self.messages = []
        self.times = []
    
    def __call__(self, text):
        self.messages += [{"role": "user", "content":text}]
        return self.llm.chat.completions.create(model="gpt-3.5-turbo", messages=self.messages, actions = [self.get_current_time])
        
    @action(name="GetCurrentTime")
    def get_current_time(self) -> str:
        """
        Use this for getting the current time in the specified time zone.
        
        :return: A string representing the current time in the specified time zone.
        """
        import datetime
        current_time = datetime.datetime.now()

        self.times += [str(current_time)]
        
        return f"The current time is {current_time}"

agent = AgentV0()


In [ ]:
# You can invoke actions just like regular instance methods
agent.get_current_time() # Output: 'The current time is 20:34.'


agent("what time is it")

**Grouping and Extending Actions Through Inheritance**

In [ ]:
class LangChainTools:
    @action(name="GoogleSearch")
    def google_search(self, query: str) -> str:
        """
        Perform a Google search using the provided query. 
        
        This action requires `langchain` and `google-api-python-client` installed, and GOOGLE_API_KEY, GOOGLE_CSE_ID environment variables.
        See https://python.langchain.com/docs/integrations/tools/google_search.

        :param query: The search query to be used for the Google search.
        :return: The search results as a string.
        """
        from langchain.utilities import GoogleSearchAPIWrapper

        search = GoogleSearchAPIWrapper()
        return search.run(query)
    
class AgentV1(AgentV0, LangChainTools):
    def __call__(self, text):
        self.messages += [{"role": "user", "content":text}]
        return self.llm.chat.completions.create(model="gpt-3.5-turbo", messages=self.messages, actions = [self.google_search])

agent = AgentV1()
agent("what happened today")

We could use parameter `orch` when calling the chat completion API. This feature will allow us for more precise control over the specific set of tools available to the LLM during each interaction.

Example:
```python
client.chat.completions.create(
    messages = ...
    actions=[a1, a2, a3], # First, LLM respond with either a1, a2 or a3, or text without action
    # Define the orchestration logic for actions:
    orch={
        a1.name: [a2, a3],  # If a1 is invoked, the next response will be either a2, a3 or a text response.
        a2.name: a3,      # If a2 is invoked, the next action will be a3
        a3.name: [a4]     # If a3 is invoked, the next response will be a4 or a text response.
        a4.name: None     # If a4 is invoked, the next response will guarantee to be a text message
    }
)
```

For details please take a look at [here](https://github.com/TengHu/ActionWeaver?tab=readme-ov-file#orchestration-of-actions-experimental )


In [34]:
class FileAgent(AgentV0):
    @action(name="FileHandler")
    def handle_file(self, instruction: str) -> str:
        """
        Handles ALL user instructions related to file operations.
    
        Args:
            instruction (str): The user's instruction about file handling.
    
        Returns:
            str: The response to the user's question.
        """
        print (f"Handling {instruction}")
        return instruction
        

    @action(name="ListFiles")
    def list_all_files_in_repo(self, repo_path: str ='.') -> List:
        """
        Lists all the files in the given repository.
    
        :param repo_path: Path to the repository. Defaults to the current directory.
        :return: List of file paths.
        """

        print(f"list_all_files_in_repo: {repo_path}")
        
        file_list = []
        for root, _, files in os.walk(repo_path):
            for file in files:
                file_list.append(os.path.join(root, file))
            break
        return file_list

    @action(name="ReadFile")
    def read_from_file(self, file_path: str) -> str:
        """
        Reads the content of a file and returns it as a string.
    
        :param file_path: The path to the file that needs to be read.
        :return: A string containing the content of the file.
        """
        print(f"read_from_file: {file_path}")
        
        with open(file_path, 'r') as file:
            content = file.read()
        return f"The file content: \n{content}"

    def __call__(self, text):
        self.messages += [{"role": "user", "content":text}]
        return self.llm.chat.completions.create(model="gpt-3.5-turbo", messages=self.messages, actions = [self.handle_file], orch = {self.handle_file.name: [self.list_all_files_in_repo, self.read_from_file]})



In [35]:
agent = FileAgent()

In [36]:
agent("Take file action [list all files in current repository]")

Handling list all files
list_all_files_in_repo: .


ChatCompletion(id='chatcmpl-8atToDzgZh5C9gh4xNWYsJtsiaNPS', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='Here are the files in the current repository:\n\n- langsmith.ipynb\n- azure_tutorial-Copy1.ipynb\n- parallel_tools.log\n- untitled.md\n- parallel_tools.ipynb\n- test.log\n- stateful_agent.ipynb\n- huggingface.ipynb\n- anyscale.ipynb\n- ReAct.ipynb\n- structured_extraction.log\n- quickstart.ipynb\n- structured_extraction.ipynb\n- azure_tutorial.ipynb\n- litellm.ipynb\n- cookbook.ipynb\n- agent.log\n- orchestration.ipynb', role='assistant', function_call=None, tool_calls=None))], created=1703803764, model='gpt-3.5-turbo-0613', object='chat.completion', system_fingerprint=None, usage=CompletionUsage(completion_tokens=130, prompt_tokens=256, total_tokens=386))